# 기본 설정

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS as stopwords
from PIL import Image, ImageFilter
from wordcloud import ImageColorGenerator
from matplotlib import font_manager, rc
from collections import Counter
import konlpy
from konlpy.tag import Mecab
import time
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from random import shuffle
import json
from tqdm.notebook import tqdm
tqdm.pandas()
import gensim 
from gensim.models import Word2Vec, FastText, Doc2Vec
import fasttext
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Loading BokehJS ...

### 함수 모음

In [2]:
def search(user_input):
    temp_list = []
    for word in word_dict:
        if user_input in word:
            temp_list.append(word)
    return temp_list

def similarWord(model, text):
    return model.most_similar(positive=[text], topn=30)

def similar_senators(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(round(similarity[i]*100, 2)) + "%"
    return result

def similar_senators_all(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {} {}'.format(senators_json[item]['한자이름'], senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    result = {}
    for i in list(rating):
        result[i] = round(similarity[i]*100, 2)
    return result


def similar_senators_with_q_all(user_input, senators_json, senators_df, word2vec_word_dict, quartile):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # quartile 안에 있는 senator 구하기
    list_senators = senators_df[senators_df['제정및전부개정법률안 비율'] > senators_df['제정및전부개정법률안 비율'].quantile(q=quartile, interpolation='nearest')]['이름'].to_list()
    # senator 찾기
    similarity = {}
    for item in senators_json:
        if item in list_senators:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
            similarity['{} {} {}'.format(senators_json[item]['한자이름'], senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
        else:
            continue
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    result = {}
    for i in list(rating):
        result[i] = round(similarity[i]*100, 2)
    return result

def similar_senators_with_q(user_input, senators_json, senators_df, word2vec_word_dict, quartile):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # quartile 안에 있는 senator 구하기
    list_senators = senators_df[senators_df['제정및전부개정법률안 비율'] > senators_df['제정및전부개정법률안 비율'].quantile(q=quartile, interpolation='nearest')]['이름'].to_list()
    # senator 찾기
    similarity = {}
    for item in senators_json:
        if item in list_senators:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
            similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
        else:
            continue
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(round(similarity[i]*100, 2)) + "%"
    return result

def show_senators_bills_contents(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_ids = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['bill_id'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_ids.extend(semi_result)
        list_temp_keyword_bills_ids = list(set(list_temp_keyword_bills_ids))
        shuffle(list_temp_keyword_bills_ids)
        list_bills_title = df_bills[df_bills['bill_id'].isin(list_temp_keyword_bills_ids[:3])]['법안명'].values.tolist()
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

def show_senators_bills_titles(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_titles = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['법안명'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_titles.extend(semi_result)
        list_temp_keyword_bills_titles = list(set(list_temp_keyword_bills_titles))
        shuffle(list_temp_keyword_bills_titles)
        list_bills_titlle = df_bills.loc(list_temp_keyword_bills_titles[:3])
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

In [3]:
def count_bills(name, bills_df):
    df_temp = bills_df[bills_df['발의자'].str.contains(name)]
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_

def count_bills_all(bills_df):
    df_temp = bills_df
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_


def get_quantile(senator):
    result = result
    return result

In [4]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

def out_stopwords(data, list_stopwords):
    data = data.split(" ")
    list_stopwords = list_stopwords
    data = [token for token in data if token not in list_stopwords]
    return data

def show_bills(bills_id, df_bills):
    for bill_id in bills_id:
        print(df_bills[df_bills['bill_id']==int(bill_id)]['법안명_법안내용'].iloc[0])
        print("*"*80)        
        
def input2vec(user_input, stopword_list, word2vec_word_dict):
    tokenized_input = tokenized_mecab(user_input)
    tokenized_input_outStopwords = out_stopwords(tokenized_input, stopword_list)
    
    list_vector = []
    for word in tokenized_input_outStopwords:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = (np.sum(list_vector, axis=0) / len(list_vector)).tolist()
    return user_vector
      
def similar_bills(user_vector, json_bills, word2vec_word_dict):
    similarity = {}
    for item in json_bills:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(json_bills[item]['vector']).reshape(1,-1))
        similarity[json_bills[item]['bill_id']] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [str(key) for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    return top_5

In [5]:
def get_precision_by_topic(usr_input):
    user_input = usr_input
    user_vector = input2vec(user_input, list_stopword, word_dict)

    dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
    df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

    df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
    df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
    df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
    df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
    df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

    df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

    df_merged['rank'] = range(1,df_merged.shape[0]+1)

    df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

    df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
    senators_list_name = df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:5, :]['name_party'].str.split(' ', 1).str[1].tolist()

    user_tokenized = tokenized_mecab(user_input)
    user_tokenized = out_stopwords(user_tokenized, list_stopword)
    user_tokenized = list(set(user_tokenized))

    bill_filter = bills[bills['법안명_법안내용'].str.contains('|'.join(user_tokenized))]
    bill_filter['snt'] = bill_filter['발의자'].str.split(' ')

    list_temp = []
    for i in bill_filter['snt'].tolist():
        list_temp.extend(i)

    dict_result = {k: v for k, v in sorted(dict(Counter(list_temp)).items(), key=lambda item: item[1], reverse=True)}

    senators_temp = senators.copy()
    senators_temp['이름/정당'] = senators_temp['한글이름'] + ' ' + senators_temp['정당']
    for idx in senators_temp.index:
        senators_temp.at[idx, 'NameParty'] = "{}({}/{})".format(senators_temp.loc[idx]['이름'].split(' ')[0], senators_temp.loc[idx]['정당'], senators_temp.loc[idx]['이름'].split(' ')[1])

    senators_temp['Count'] = senators_temp['NameParty'].map(dict_result)

    boolean_list = senators_temp[senators_temp['Count']>=senators_temp['Count'].median()]['이름/정당'].tolist()

    return accuracy(senators_list_name, boolean_list)

In [6]:
def get_score_by_topic(usr_input):
    user_input = usr_input
    user_vector = input2vec(user_input, list_stopword, word_dict)

    # Get senator score
    dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
    df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

    df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
    df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
    df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
    df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
    df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

    df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

    df_merged['rank'] = range(1,df_merged.shape[0]+1)

    df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

    df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
    df_merged['Snt/Party/NameHan'] = df_merged['name_party'].str.split(' ').str[1] + "(" + df_merged['name_party'].str.split(' ').str[2] + '/' + df_merged['name_party'].str.split(' ').str[0] + ")"
    # Filter bills
    user_tokenized = tokenized_mecab(user_input)
    user_tokenized = out_stopwords(user_tokenized, list_stopword)
    user_tokenized = list(set(user_tokenized))

    bill_filter = bills[bills['법안명_법안내용'].str.contains('|'.join(user_tokenized))]
    bill_filter['snt'] = bill_filter['발의자'].str.split(' ')

    list_temp = []
    for i in bill_filter['snt'].tolist():
        list_temp.extend(i)
    list_temp = [snt for snt in  list_temp if snt in senators['Snt/Party/NameHan'].tolist()]

    # 관련 법안을 발의한 사람들
    actual = {k: v for k, v in sorted(dict(Counter(list_temp)).items(), key=lambda item: item[1], reverse=True)}

    predicted = df_merged[['Snt/Party/NameHan', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:len(actual), :]['Snt/Party/NameHan'].tolist()

    recall = len([snt for snt in predicted if snt in actual.keys()])/len(actual)
    precision = len([snt for snt in predicted if snt in actual.keys()]) / len(predicted)
    f1_score = 2*(recall * precision) / (recall + precision)
    return {"recall":recall, "precision" : precision, "f1_score": f1_score}

In [7]:
bills = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차 데이터 및 jupyter/bills_24839_all_cleansed.csv')
senators = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차 데이터 및 jupyter/(원본) assembly_memebers_295.csv')

In [8]:
senators['한글이름'] = senators['이름'].str.split(' ').str[0]
senators['한자이름'] = senators['이름'].str.split(' ').str[1]

In [9]:
senators['Snt/Party/NameHan'] = senators['이름'].str.split(' ').str[0] + "(" + senators['정당'].str.split(' ').str[0] + "/" + senators['이름'].str.split(' ').str[1] + ")"

In [10]:
bills = bills[bills['발의자'] != '위원장']
bills = bills[bills['발의자'] != '정부'] 
bills = bills.dropna()

In [11]:
embedding_model = Word2Vec.load('../models/word2vec ver21.0.model')

In [12]:
with open('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차 데이터 및 jupyter/senators ver21.0.json', 'r') as f:
    senators_json = json.load(f)

In [13]:
with open('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차 데이터 및 jupyter/bills ver21.0.json', 'r') as f:
    bills_json = json.load(f)

In [14]:
word_dict = {}
for vocab, vector in zip(embedding_model.wv.index2word, embedding_model.wv.vectors):
    word_dict[vocab] = vector

# 전체 distribution 분포 살펴보기

In [16]:
dict_bills_portion = {}
for senator in senators['한글이름']:
    dict_bills_portion[senator] = count_bills(senator, bills)

In [17]:
df_bills_portion = pd.DataFrame(dict_bills_portion).T

In [18]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].str[:-1]
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].str[:-1]
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].str[:-1]
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].str[:-1]

In [19]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].astype(float)
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].astype(float)
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].astype(float)
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].astype(float)

In [20]:
df_bills_portion['제정및전부개정법률안 비율'] = df_bills_portion['제정법률안 비율'] + df_bills_portion['전부개정법률안 비율']

# 상위 % 잘라낸 뒤 추천하기

In [21]:
df_bills_portion.reset_index(inplace=True)

In [22]:
senators = senators.merge(df_bills_portion, left_on="한글이름", right_on="index", how="left")
senators = senators.drop(columns=['index'])

In [23]:
df_temp = senators[['이름', '한글이름','제정및전부개정법률안 비율']].sort_values(['제정및전부개정법률안 비율'])

In [24]:
df_temp['순위'] = range(1, df_temp.shape[0]+1)

In [25]:
df_temp['percentile'] = round(100-(df_temp['순위']/295) * 100, 5)

In [26]:
df_temp['percentile'] = df_temp['percentile'].astype(str)
df_temp['percentile'] = df_temp['percentile'] + '%'
df_temp

,이름,한글이름,제정및전부개정법률안 비율,순위,percentile
284,허윤정 許允貞,허윤정,0.00,1,99.66102%
245,정은혜 鄭恩惠,정은혜,0.00,2,99.32203%
248,정점식 鄭点植,정점식,2.63,3,98.98305%
271,최도자 崔道子,최도자,3.89,4,98.64407%
224,임재훈 林哉勳,임재훈,4.32,5,98.30508%
14,권칠승 權七勝,권칠승,4.38,6,97.9661%
181,이규희 李揆熙,이규희,4.41,7,97.62712%
220,이후삼 李厚三,이후삼,4.42,8,97.28814%
15,금태섭 琴泰燮,금태섭,4.44,9,96.94915%
136,신용현 申容賢,신용현,4.49,10,96.61017%


# 의원별 입법지수 계산

In [27]:
def count_senators(data):
    data = [x for x in data if '/' in x]
    return len(data)

In [28]:
dict_count_biils = {}
for row in senators.itertuples():
    dict_temp = {}
    dict_temp['한글이름'] = row[-7]
    
    # Get the dataframe
    df_temp = bills[bills['발의자'].str.contains(row[-7])]
    bills_전부개정법률안 = df_temp[df_temp['법안명'].str.contains('전부개정법률안')]
    bills_제정법안 = df_temp[(~df_temp['법안명'].str.contains('일부개정법률안'))& (~df_temp['법안명'].str.contains('전부개정법률안'))]
    bills_total = pd.concat([bills_제정법안, bills_전부개정법률안])
    
    # count mean senators proposing 
    bills_total['발의자 수'] = bills_total['발의자'].str.split(' ').map(count_senators)
    
    # save in dictionary
    dict_temp['전체 발의법안 수'] = bills_total.shape[0]
    if dict_temp['전체 발의법안 수'] == 0:
        dict_temp['평균 발의자 수'] = 0.000000000000009
    else:    
        dict_temp['평균 발의자 수'] = round(bills_total['발의자 수'].mean())

    dict_count_biils[row[-6]] = dict_temp

In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [53]:
pd.DataFrame(dict_count_biils).T

,한글이름,전체 발의법안 수,평균 발의자 수
강길부(무소속/姜吉夫),姜吉夫,24,64
강병원(더불어민주당/姜炳遠),姜炳遠,95,38
강석진(미래통합당/姜錫振),姜錫振,61,47
강석호(미래통합당/姜碩鎬),姜碩鎬,61,48
강창일(더불어민주당/姜昌一),姜昌一,76,39
강효상(미래통합당/姜孝祥),姜孝祥,35,67
강훈식(더불어민주당/姜勳植),姜勳植,118,33
경대수(미래통합당/慶大秀),慶大秀,66,44
고용진(더불어민주당/高榕禛),高榕禛,78,33
곽대훈(미래통합당/郭大勳),郭大勳,74,46


In [30]:
df_result = pd.DataFrame(dict_count_biils).T.reset_index().rename(columns={'index':'한자이름'})
df_result['법안수/평균 발의자 수'] = df_result['전체 발의법안 수'] / df_result['평균 발의자 수'] 
df_result.sort_values(by='법안수/평균 발의자 수')
df_result['bills_index'] = df_result['법안수/평균 발의자 수'].astype(float)
df_result['bills_index_scaled'] = scaler.fit_transform(df_result[['bills_index']])

In [31]:
df_result

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,강길부(무소속/姜吉夫),姜吉夫,24,64,0.375,0.375000,0.034539
1,강병원(더불어민주당/姜炳遠),姜炳遠,95,38,2.5,2.500000,0.230263
2,강석진(미래통합당/姜錫振),姜錫振,61,47,1.29787,1.297872,0.119541
3,강석호(미래통합당/姜碩鎬),姜碩鎬,61,48,1.27083,1.270833,0.117050
4,강창일(더불어민주당/姜昌一),姜昌一,76,39,1.94872,1.948718,0.179487
5,강효상(미래통합당/姜孝祥),姜孝祥,35,67,0.522388,0.522388,0.048115
6,강훈식(더불어민주당/姜勳植),姜勳植,118,33,3.57576,3.575758,0.329346
7,경대수(미래통합당/慶大秀),慶大秀,66,44,1.5,1.500000,0.138158
8,고용진(더불어민주당/高榕禛),高榕禛,78,33,2.36364,2.363636,0.217703
9,곽대훈(미래통합당/郭大勳),郭大勳,74,46,1.6087,1.608696,0.148169


In [32]:
bills_index = df_result.copy()

In [52]:
bills_index

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,강길부(무소속/姜吉夫),姜吉夫,24,64,0.375,0.375000,0.034539
1,강병원(더불어민주당/姜炳遠),姜炳遠,95,38,2.5,2.500000,0.230263
2,강석진(미래통합당/姜錫振),姜錫振,61,47,1.29787,1.297872,0.119541
3,강석호(미래통합당/姜碩鎬),姜碩鎬,61,48,1.27083,1.270833,0.117050
4,강창일(더불어민주당/姜昌一),姜昌一,76,39,1.94872,1.948718,0.179487
5,강효상(미래통합당/姜孝祥),姜孝祥,35,67,0.522388,0.522388,0.048115
6,강훈식(더불어민주당/姜勳植),姜勳植,118,33,3.57576,3.575758,0.329346
7,경대수(미래통합당/慶大秀),慶大秀,66,44,1.5,1.500000,0.138158
8,고용진(더불어민주당/高榕禛),高榕禛,78,33,2.36364,2.363636,0.217703
9,곽대훈(미래통합당/郭大勳),郭大勳,74,46,1.6087,1.608696,0.148169


# Change index to percentage

In [33]:
url= "/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차_stopwords 추가/stopwords ver1.0.txt"
stopwords_file = list(open(url,"r"))
list_stopword = []
for i in stopwords_file:
    list_stopword.append(i[:-1])

### 예시 1

In [51]:
df_similarity

,name_party,similarity,한자이름,similarity_scaled
0,金武星 김무성 미래통합당,77.93,金武星,1.000000
1,金順禮 김순례 미래통합당,77.92,金順禮,0.998498
2,朴炳錫 박병석 더불어민주당,77.89,朴炳錫,0.993994
3,文喜相 문희상 무소속,77.80,文喜相,0.980480
4,鄭柄國 정병국 미래통합당,77.80,鄭柄國,0.980480
5,李相敦 이상돈 무소속,77.78,李相敦,0.977477
6,李賢在 이현재 무소속,77.70,李賢在,0.965465
7,庾敏鳳 유민봉 미래통합당,77.69,庾敏鳳,0.963964
8,沈載權 심재권 더불어민주당,77.69,沈載權,0.963964
9,河泰慶 하태경 미래통합당,77.68,河泰慶,0.962462


In [34]:
user_input='저는 장애인 아이를 키우고 있는 부모입니다. 아이가 더 좋은 복지서비스를 받아 한 사회의 구성원으로서 살아갔으면 합니다. 특수교육과 장애인복지시설 등에 관심이 있습니다'
user_vector = input2vec(user_input, list_stopword, word_dict)

dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

df_merged['rank'] = range(1,df_merged.shape[0]+1)

df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']

In [50]:
df_merged

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time
0,金武星 김무성 미래통합당,77.93,1.000000,NaN,1,0.996610,0.996610
1,金順禮 김순례 미래통합당,77.92,0.998498,NaN,2,0.993220,0.991729
2,朴炳錫 박병석 더불어민주당,77.89,0.993994,NaN,3,0.989831,0.983886
3,文喜相 문희상 무소속,77.80,0.980480,NaN,4,0.986441,0.967186
4,鄭柄國 정병국 미래통합당,77.80,0.980480,NaN,5,0.983051,0.963862
5,李相敦 이상돈 무소속,77.78,0.977477,NaN,6,0.979661,0.957597
6,李賢在 이현재 무소속,77.70,0.965465,NaN,7,0.976271,0.942556
7,庾敏鳳 유민봉 미래통합당,77.69,0.963964,NaN,8,0.972881,0.937823
8,沈載權 심재권 더불어민주당,77.69,0.963964,NaN,9,0.969492,0.934555
9,河泰慶 하태경 미래통합당,77.68,0.962462,NaN,10,0.966102,0.929837


In [35]:
df_merged

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time
0,金武星 김무성 미래통합당,77.93,1.000000,NaN,1,0.996610,0.996610
1,金順禮 김순례 미래통합당,77.92,0.998498,NaN,2,0.993220,0.991729
2,朴炳錫 박병석 더불어민주당,77.89,0.993994,NaN,3,0.989831,0.983886
3,文喜相 문희상 무소속,77.80,0.980480,NaN,4,0.986441,0.967186
4,鄭柄國 정병국 미래통합당,77.80,0.980480,NaN,5,0.983051,0.963862
5,李相敦 이상돈 무소속,77.78,0.977477,NaN,6,0.979661,0.957597
6,李賢在 이현재 무소속,77.70,0.965465,NaN,7,0.976271,0.942556
7,庾敏鳳 유민봉 미래통합당,77.69,0.963964,NaN,8,0.972881,0.937823
8,沈載權 심재권 더불어민주당,77.69,0.963964,NaN,9,0.969492,0.934555
9,河泰慶 하태경 미래통합당,77.68,0.962462,NaN,10,0.966102,0.929837


In [36]:
df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity_scaled,bills_quantile,quality_index_time
0,金武星 김무성 미래통합당,1.000000,0.996610,0.996610
1,金順禮 김순례 미래통합당,0.998498,0.993220,0.991729
2,朴炳錫 박병석 더불어민주당,0.993994,0.989831,0.983886
3,文喜相 문희상 무소속,0.980480,0.986441,0.967186
4,鄭柄國 정병국 미래통합당,0.980480,0.983051,0.963862
5,李相敦 이상돈 무소속,0.977477,0.979661,0.957597
6,李賢在 이현재 무소속,0.965465,0.976271,0.942556
7,庾敏鳳 유민봉 미래통합당,0.963964,0.972881,0.937823
8,沈載權 심재권 더불어민주당,0.963964,0.969492,0.934555
9,河泰慶 하태경 미래통합당,0.962462,0.966102,0.929837


In [37]:
senators_list = df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:5, :]['name_party'].str.split(' ', 1).str[0].tolist()
senators_list_name = df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:5, :]['name_party'].str.split(' ', 1).str[1].tolist()

In [38]:
similarity = {}
for id_ in bills[bills['발의자'].str.contains('|'.join(senators_list))]['bill_id'].tolist():
    if str(id_) in bills_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(bills_json[str(id_)]['vector']).reshape(1,-1))
    similarity[id_] = float(sim)
similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
top_5 = rating[:5]

result = {}
for i in top_5:
    result[i] = str(abs(round((similarity[i]*100), 2))) + "%"

In [39]:
bills[bills['bill_id'].isin(result)]

,bill_id,입안일,법안명,법안내용,발의자,상태,법안명_이름만,법안명_법안내용
4706,4706,2019-04-11,아이돌봄 지원법 일부개정법률안(신보라의원 등 15인),최근 아이돌보미가 14개월 된 아동을 학대하는 사건이 발생함에 따라 아이돌보미에 ...,신보라(자유한국당/申普羅) 강효상(자� f63 �한국당/姜孝祥) 곽대훈(자유한국당/...,계류,아이돌봄 지원법 일부개정법률안,아이돌봄 지원법 일부개정법률안 최근 아이돌보미가 14개월 된 아동을 학대하는 사건...
10267,10267,2018-06-25,아동복지법 일부개정법률안(김순례의원 등 10인),"핵가족화, 여성 경제활동 증가 등 양육환경 변화로 돌봄에 대한 사회적 요구가 증가...",김순례(자유한국당/金順禮) 강석진(자유한국당/姜錫振) 김선동(자유한국당/金善東) 김...,대안반영폐기,아동복지법 일부개정법률안,"아동복지법 일부개정법률안 핵가족화, 여성 경제활동 증가 등 양육환경 변화로 돌봄에..."
10786,10786,2018-05-04,초·중등교육법 일부개정법률안(남인순의원 등 12인),"초등학교 취학 이후 돌봄 공백은 학부모, 특히 여성들이 사회참여와 소득활동을 포기...",남인순(더불어민주당/南仁順) 금태섭(더불어민주당/琴泰燮) 김상희(더불어민주당/金相姬...,계류,초·중등교육법 일부개정법률안,"초·중등교육법 일부개정법률안 초등학교 취학 이후 돌봄 공백은 학부모, 특히 여성들..."
13400,13400,2017-12-11,아이돌봄 지원법 일부개정법률안(원유철의원 등 10인),현행법은 가정의 아이돌봄을 지원하기 위하여 일정한 자격과 기준을 갖춘 아이돌보미와...,원유철(자유한국당/元裕哲) 김순례(자유한국당/金順禮) 김한표(자유한국당/金漢杓) 나...,계류,아이돌봄 지원법 일부개정법률안,아이돌봄 지원법 일부개정법률안 현행법은 가정의 아이돌봄을 지원하기 위하여 일정한 ...
13453,13453,2017-12-08,아이돌봄 지원법 일부개정법률안(원유철의원 등 10인),현행법은 취업한 보호자의 일·가정 양립 등을 지원하기 위하여 아이돌봄서비스 제공기...,원유철(자유한국당/元裕哲) 박덕흠(자유한국당/朴德欽) 박찬우(자유한국당/朴贊佑) 서...,계류,아이돌봄 지원법 일부개정법률안,아이돌봄 지원법 일부개정법률안 현행법은 취업한 보호자의 일·가정 양립 등을 지원하...


## Accuracy

In [40]:
user_tokenized = tokenized_mecab(user_input)
user_tokenized = out_stopwords(user_tokenized, list_stopword)
user_tokenized = list(set(user_tokenized))

In [41]:
bill_filter = bills[bills['법안명_법안내용'].str.contains('|'.join(user_tokenized))]
bill_filter['snt'] = bill_filter['발의자'].str.split(' ')

<ipython-input-41-0b869930831a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')


In [42]:
list_temp = []
for i in bill_filter['snt'].tolist():
    list_temp.extend(i)

In [43]:
dict_result = {k: v for k, v in sorted(dict(Counter(list_temp)).items(), key=lambda item: item[1], reverse=True)}

In [44]:
senators_temp = senators.copy()
senators_temp['이름/정당'] = senators_temp['한글이름'] + ' ' + senators_temp['정당']
for idx in senators_temp.index:
    senators_temp.at[idx, 'NameParty'] = "{}({}/{})".format(senators_temp.loc[idx]['이름'].split(' ')[0], senators_temp.loc[idx]['정당'], senators_temp.loc[idx]['이름'].split(' ')[1])

In [45]:
senators_temp['Count'] = senators_temp['NameParty'].map(dict_result)

In [46]:
senators_list_name

['김무성 미래통합당', '김순례 미래통합당', '박병석 더불어민주당', '문희상 무소속', '정병국 미래통합당']

In [47]:
boolean_list = senators_temp[senators_temp['Count']>=senators_temp['Count'].median()]['이름/정당'].tolist()

In [48]:
def accuracy(snt_list, boolean_list):
    count = 0
    for snt in snt_list:
        if snt in boolean_list:
            count += 1
            
    return str(count*20) + '%'

In [49]:
accuracy(senators_list_name, boolean_list)

'0%'

### 예시 2

In [49]:
user_input='저는 대학을 졸업한 청년입니다. 대학 졸업하고 소셜 벤처를 창업하고자 합니다. 소셜 벤처와 청년창업을 지원해주셨으면 합니다.'
user_vector = input2vec(user_input, list_stopword, word_dict)

dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

df_merged['rank'] = range(1,df_merged.shape[0]+1)

df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']

In [50]:
df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity_scaled,bills_quantile,quality_index_time
0,鄭恩惠 정은혜 더불어민주당,1.000000,0.996610,0.996610
1,丁世均 정세균 더불어민주당,0.846361,0.993220,0.840623
2,李相珉 이상민 더불어민주당,0.843666,0.989831,0.835086
3,全希卿 전희경 미래통합당,0.843666,0.986441,0.832226
4,鄭仁和 정인화 무소속,0.834232,0.983051,0.820092
5,鄭点植 정점식 미래통합당,0.815364,0.979661,0.798780
6,尹一逵 윤일규 더불어민주당,0.815364,0.976271,0.796016
7,卞在一 변재일 더불어민주당,0.811321,0.972881,0.789319
8,洪文鐘 홍문종 무소속,0.809973,0.969492,0.785262
9,崔然惠 최연혜 미래통합당,0.804582,0.966102,0.777308


In [51]:
senators_list = df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:5, :]['name_party'].str.split(' ', 1).str[0].tolist()
senators_list_name = df_merged[['name_party', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:5, :]['name_party'].str.split(' ', 1).str[1].tolist()

In [52]:
similarity = {}
for id_ in bills[bills['발의자'].str.contains('|'.join(senators_list))]['bill_id'].tolist():
    if str(id_) in bills_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(bills_json[str(id_)]['vector']).reshape(1,-1))
    similarity[id_] = float(sim)
similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
top_5 = rating[:5]

result = {}
for i in top_5:
    result[i] = str(abs(round((similarity[i]*100), 2))) + "%"

In [53]:
bills[bills['bill_id'].isin(result)]

,bill_id,입안일,법안명,법안내용,발의자,상태,법안명_이름만,법안명_법안내용
1102,1102,2019-10-31,벤처기업육성에 관한 특별조치법 일부개정법률안(서영교의원 등 13인),재외국민이 외국에서 벤처기업을 창업하는 경우 관련 정보 및 사업장이 부족하여 창업...,서영교(더불어민주당/徐瑛敎) 김경협(더불어민주당/金炅俠) 김두관(더불어민주당/金斗官...,계류,벤처기업육성에 관한 특별조치법 일부개정법률안,벤처기업육성에 관한 특별조치법 일부개정법률안 재외국민이 외국에서 벤처기업을 창업하...
12837,12837,2018-01-10,조세특례제한법 일부개정법률안(박준영의원 등 20인),현행법은 청년실업 문제를 해결하기 위해 15세 이상 29세 이하인 청년이 중소기업...,박준영(국민의당/朴晙瑩) 김경협(더불어민주당/金炅俠) 김관영(국민의당/金寬永) 김수...,대안반영폐기,조세특례제한법 일부개정법률안,조세특례제한법 일부개정법률안 현행법은 청년실업 문제를 해결하기 위해 15세 이상 ...
21037,21037,2016-10-24,청년 첫일자리 지원 특별법안(박홍근의원 등 15인),원하는 일자리에 대한 눈높이와 현실적으로 얻을 수 있는 일자리 사이의 미스매치(m...,박홍근(더불어민주당/朴洪根) 김경수(더불어민주당/金慶洙) 김현권(더불어민주당/金玄權...,계류,청년 첫일자리 지원 특별법안,청년 첫일자리 지원 특별법안 원하는 일자리에 대한 눈높이와 현실적으로 얻을 수 있...
21064,21064,2016-10-24,청년 창업 활성화 및 청년창업기업 지원에 관한 특별법안(김경수의원 등 16인),"경제성장률의 둔화, 자동화 등 구조적 일자리 감소에 따라 국가성장의 패러다임이 기...",김경수(더불어민주당/金慶洙) 김병관(더불어민주당/金炳官) f87 김현권(더불어민주당...,계류,청년 창업 활성화 및 청년창업기업 지원에 관한 특별법안,"청년 창업 활성화 및 청년창업기업 지원에 관한 특별법안 경제성장률의 둔화, 자동화..."
21333,21333,2016-10-05,조세특례제한법 일부개정법률안(조경태의원 등 10인),"기업의 고용창출능력 저하 등으로 청년실업률이 해마다 높아지고 있는 가운데, 청년들...",조경태(새누리당/趙慶泰) 김재경(새누리당/金在庚) 박완수(새누리당/朴完洙) 박준영(...,대안반영폐기,조세특례제한법 일부개정법률안,조세특례제한법 일부개정법률안 기업의 고용창출능력 저하 등으로 청년실업률이 해마다 ...


## Accuracy

In [54]:
user_tokenized = tokenized_mecab(user_input)
user_tokenized = out_stopwords(user_tokenized, list_stopword)
user_tokenized = list(set(user_tokenized))

In [55]:
bill_filter = bills[bills['법안명_법안내용'].str.contains('|'.join(user_tokenized))]
bill_filter['snt'] = bill_filter['발의자'].str.split(' ')

<ipython-input-55-0b869930831a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')


In [56]:
user_tokenized

['청년', '대학', '주', '셨', '으면', '창업', '지원', '저', '졸업', '고자', '소셜', '벤처']

In [57]:
list_temp = []
for i in bill_filter['snt'].tolist():
    list_temp.extend(i)

In [58]:
dict_result = {k: v for k, v in sorted(dict(Counter(list_temp)).items(), key=lambda item: item[1], reverse=True)}

In [59]:
senators_temp = senators.copy()
senators_temp['이름/정당'] = senators_temp['한글이름'] + ' ' + senators_temp['정당']
for idx in senators_temp.index:
    senators_temp.at[idx, 'NameParty'] = "{}({}/{})".format(senators_temp.loc[idx]['이름'].split(' ')[0], senators_temp.loc[idx]['정당'], senators_temp.loc[idx]['이름'].split(' ')[1])

In [60]:
senators_temp['Count'] = senators_temp['NameParty'].map(dict_result)

In [61]:
boolean_list = senators_temp[senators_temp['Count']>=senators_temp['Count'].median()]['이름/정당'].tolist()

In [62]:
accuracy(senators_list_name, boolean_list)

'20%'

# Precision by topic

## 원자력

In [63]:
dict_result = {}
dict_result["친환경 건축물"] = get_precision_by_topic('녹색조형물 에너지 성능 공공')
dict_result["특수교육"] = get_precision_by_topic('특수교육 장애인 복지서비스 장애인복지시설')
dict_result["노인학대"] = get_precision_by_topic('노인학대 예방 복지')
dict_result["성범죄"] = get_precision_by_topic('성범죄 처벌 강화 성폭력')
dict_result["소방공무원"] = get_precision_by_topic('소방공무원 안전 보건')
dict_result["장애인 복지"] = get_precision_by_topic('장애인 복지 일자리')
dict_result["벤처기업/중소기업/창업"] = get_precision_by_topic('벤처기업 중소기업 창업 지원 청년')
dict_result["낙태죄 폐지"] = get_precision_by_topic('낙태죄 폐지 건강권 자기결정권')
dict_result["임대차 보호"] =  get_precision_by_topic('임대차 보호')
dict_result["직장 내 성희롱 예방"] = get_precision_by_topic('직장 내 성희롱 예방')
dict_result["신재생에너지 확대"] = get_precision_by_topic('신에너지 재생에너지 안정 공급 확대 활성화')
dict_result["난민"] = get_precision_by_topic('난민 면접 심사')
dict_result["특수임무 유공자"] = get_precision_by_topic('특수임무유공자 예우')
dict_result["저탄소 성장"] = get_precision_by_topic('저탄소 녹색성장 에너지 확보')
dict_result["사회적참사 진상규명 및 안전사회"] = get_precision_by_topic('사회적참사 징역 벌칙 벌금')

<ipython-input-5-b95c6e1b0828>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-5-b95c6e1b0828>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-5-b95c6e1b0828>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [64]:
get_precision_by_topic('사회적참사 징역 벌칙 벌금')

<ipython-input-5-b95c6e1b0828>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')


'40%'

In [65]:
dict_result

{'친환경 건축물': '0%',
 '특수교육': '0%',
 '노인학대': '0%',
 '성범죄': '20%',
 '소방공무원': '0%',
 '장애인 복지': '0%',
 '벤처기업/중소기업/창업': '20%',
 '낙태죄 폐지': '20%',
 '임대차 보호': '0%',
 '직장 내 성희롱 예방': '0%',
 '신재생에너지 확대': '20%',
 '난민': '0%',
 '특수임무 유공자': '0%',
 '저탄소 성장': '20%',
 '사회적참사 진상규명 및 안전사회': '40%'}

In [66]:
list_ = [int(i[:-1]) for i in list(dict_result.values())]
sum(list_)/len(list_)

9.333333333333334

In [67]:
len(list_)

15

# Precision, F1, Recall by topic

In [235]:
def get_score_by_topic(usr_input):
    user_input = usr_input
    user_vector = input2vec(user_input, list_stopword, word_dict)

    # Get senator score
    dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
    df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

    df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
    df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
    df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
    df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
    df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

    df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

    df_merged['rank'] = range(1,df_merged.shape[0]+1)

    df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

    df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
    df_merged['Snt/Party/NameHan'] = df_merged['name_party'].str.split(' ').str[1] + "(" + df_merged['name_party'].str.split(' ').str[2] + '/' + df_merged['name_party'].str.split(' ').str[0] + ")"
    # Filter bills
    user_tokenized = tokenized_mecab(user_input)
    user_tokenized = out_stopwords(user_tokenized, list_stopword)
    user_tokenized = list(set(user_tokenized))

    bill_filter = bills[bills['법안명_법안내용'].str.contains('|'.join(user_tokenized))]
    bill_filter['snt'] = bill_filter['발의자'].str.split(' ')

    list_temp = []
    for i in bill_filter['snt'].tolist():
        list_temp.extend(i)
    list_temp = [snt for snt in  list_temp if snt in senators['Snt/Party/NameHan'].tolist()]

    # 관련 법안을 발의한 사람들
    actual = {k: v for k, v in sorted(dict(Counter(list_temp)).items(), key=lambda item: item[1], reverse=True)}

    predicted = df_merged[['Snt/Party/NameHan', 'similarity_scaled', 'bills_quantile', 'quality_index_time']].sort_values(by=['quality_index_time'], ascending=False).iloc[:len(actual), :]['Snt/Party/NameHan'].tolist()

    recall = len([snt for snt in predicted if snt in actual.keys()])/(len(actual)*1)
    precision = len([snt for snt in predicted if snt in actual.keys()]) / (len(predicted)*1)
    f1_score = 2*(recall * precision) / (recall + precision)
    return {"recall":recall, "precision" : precision, "f1_score": f1_score}

In [292]:
dict_result = {}
# dict_result["친환경 건축물"] = get_score_by_topic('녹색조형물 에너지 성능 공공')
# dict_result["특수교육"] = get_score_by_topic('특수교육 장애인 복지서비스 장애인복지시설')
# dict_result["노인학대"] = get_score_by_topic('노인학대 예방 복지')
# dict_result["성범죄"] = get_score_by_topic('성범죄 처벌 강화 성폭력')
# dict_result["소방공무원"] = get_score_by_topic('소방공무원 안전 보건')
# dict_result["장애인 복지"] = get_score_by_topic('장애인 복지 일자리')
# dict_result["벤처기업/중소기업/창업"] = get_score_by_topic('벤처기업 중소기업 창업 지원 청년')
dict_result["낙태죄 폐지"] = get_score_by_topic('낙태죄 폐지 건강권 자기결정권 여성')
# dict_result["임대차 보호"] =  get_score_by_topic('임대차 보호')
dict_result["직장 내 성희롱 예방"] = get_score_by_topic('직장 내 성희롱 예방 여성 보호')
dict_result["신재생에너지 확대"] = get_score_by_topic('신에너지 재생에너지 안정 공급 확대 활성화')
# dict_result["난민"] = get_score_by_topic('난민 면접 심사')
# dict_result["특수임무 유공자"] = get_score_by_topic('특수임무유공자 예우')
dict_result["저탄소 성장"] = get_score_by_topic('저탄소 녹색성장 에너지 확보')
dict_result["사회적참사 진상규명 및 안전사회"] = get_score_by_topic('사회적참사 진상규명 강화 처벌 안전사회')
dict_result["어려운 표현 쉬운 표현으로"] = get_score_by_topic('표현 이해 국민 용어')
dict_result["군인 인권보호"] = get_score_by_topic('군인 인권 보호 침해')
dict_result["범죄인 인도"] = get_score_by_topic('범죄인 인도법')
dict_result["폐기물 관리 및 처리"] = get_score_by_topic('폐기물 관리')
dict_result["영화 콘텐츠 산업 진흥"] = get_score_by_topic('영화 비디오물 진흥')

<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [296]:
dict_result2 = {}
# dict_result2["친환경 건축물"] = get_score_by_topic('녹색조형물 에너지 성능 공공')
# dict_result2["특수교육"] = get_score_by_topic('특수교육 장애인 복지서비스 장애인복지시설')
# dict_result2["노인학대"] = get_score_by_topic('노인학대 예방 복지')
# dict_result2["성범죄"] = get_score_by_topic('성범죄 처벌 강화 성폭력')
# dict_result2["소방공무원"] = get_score_by_topic('소방공무원 안전 보건')
# dict_result2["장애인 복지"] = get_score_by_topic('장애인 복지 일자리')
# dict_result2["벤처기업/중소기업/창업"] = get_score_by_topic('벤처기업 중소기업 창업 지원 청년')
# dict_result2["낙태죄 폐지"] = get_score_by_topic('낙태죄 폐지 건강권 자기결정권 여성')
# dict_result2["임대차 보호"] =  get_score_by_topic('임대차 보호')
dict_result2["직장 내 성희롱 예방"] = get_score_by_topic('직장 내 성희롱 예방 여성 보호')
dict_result2["신재생에너지 확대"] = get_score_by_topic('신에너지 재생에너지 안정 공급 확대 활성화')
# dict_result2["난민"] = get_score_by_topic('난민 면접 심사')
# dict_result2["특수임무 유공자"] = get_score_by_topic('특수임무유공자 예우')
# dict_result2["저탄소 성장"] = get_score_by_topic('저탄소 녹색성장 에너지 확보')
dict_result2["사회적참사 진상규명 및 안전사회"] = get_score_by_topic('사회적참사 진상규명 강화 처벌 안전사회')
dict_result2["어려운 표현 쉬운 표현으로"] = get_score_by_topic('표현 이해 국민 용어')
dict_result2["군인 인권보호"] = get_score_by_topic('군인 인권 보호 침해')
dict_result2["범죄인 인도"] = get_score_by_topic('범죄인 인도법')
dict_result2["폐기물 관리 및 처리"] = get_score_by_topic('폐기물 관리')
dict_result2["영화 콘텐츠 산업 진흥"] = get_score_by_topic('영화 비디오물 진흥')

<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_filter['snt'] = bill_filter['발의자'].str.split(' ')
<ipython-input-235-edad1a89efa8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [298]:
df_result = pd.DataFrame(dict_result).T
df_result

,recall,precision,f1_score
낙태죄 폐지,0.653061,0.653061,0.653061
직장 내 성희롱 예방,0.716346,0.716346,0.716346
신재생에너지 확대,0.702970,0.702970,0.702970
저탄소 성장,0.754902,0.754902,0.754902
사회적참사 진상규명 및 안전사회,0.651515,0.651515,0.651515
어려운 표현 쉬운 표현으로,0.680000,0.680000,0.680000
군인 인권보호,0.669014,0.669014,0.669014
범죄인 인도,0.692308,0.692308,0.692308
폐기물 관리 및 처리,0.693878,0.693878,0.693878
영화 콘텐츠 산업 진흥,0.679803,0.679803,0.679803


In [301]:
df_result.mean()

recall       0.68938
precision    0.68938
f1_score     0.68938
dtype: float64